In [ ]:
import os
import openai

import pyaudio
import wave
import keyboard
import time

import pyttsx3

openai.api_key = os.getenv("OPENAI_API_KEY")

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

In [ ]:
def start_recording():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    frames = []

    print("Recording...")

    while keyboard.is_pressed('r'):
        data = stream.read(CHUNK)
        frames.append(data)

    return audio, stream, frames

In [ ]:
def stop_recording(audio, stream, frames):
    print("Stopped recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()

    file_path = "tmp\\test.wav"
    wf = wave.open(file_path, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    print(f"Saved audio to {file_path}")

In [ ]:
def convert_audio_to_text(audio_file):
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    return transcript.text

In [ ]:
def generate_answer(messages):
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    return completion.choices[0].message.content

In [ ]:
def text_to_speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()
    time.sleep(1)

In [ ]:
print("Press and hold the 'r' key to start recording. Release the key to stop recording.")

messages=[
        {"role": "user", "content": "Hello!"},
        {"role": "assistant", "content": "Hello, my name is NAOqi!"}]

while True:
    if keyboard.is_pressed('r'):
        audio, stream, frames = start_recording()
        while keyboard.is_pressed('r'):
            time.sleep(0.1)
        stop_recording(audio, stream, frames)
        audio_file = open("tmp\\test.wav", "rb")
        messages.append({"role": "user", "content": convert_audio_to_text(audio_file)})
        print(f"User: {messages[-1]['content']}\n")
        messages.append({"role": "assistant", "content": generate_answer(messages)})
        print(f"AI: {messages[-1]['content']}\n")
        text_to_speech(messages[-1]['content'])
    if keyboard.is_pressed('q'):
        break